In [ ]:
%pip install pydicom
%pip install customtkinter


In [ ]:
%pip install matplotlib


In [ ]:
import matplotlib
print(matplotlib.__file__)

In [ ]:
%pip install PIL
%pip install numpy
%pip install skimage

In [ ]:
%pip install scipy

In [ ]:
%pip uninstall scikit-image
%pip install scikit-image

In [ ]:
%pip install --upgrade scikit-image


In [2]:
import skimage
print(skimage.__version__)


0.20.0


In [ ]:
%pip install tkinter-tooltip

In [6]:
%pip install trimesh

     -------------------------------------- 681.1/681.1 kB 5.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
%pip install pymeshlab

     --------------------------------------- 37.3/37.3 MB 28.5 MB/s eta 0:00:00
     ---------------------------------------- 1.8/1.8 MB 28.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [Errno 13] Permission denied: 'c:\\Python311\\concrt140.dll'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
%pip install customtkinter-tooltip

In [1]:
from matplotlib import patches
import pydicom
import trimesh
import os
import tkinter as tk
from scipy.ndimage import laplace
import customtkinter
from tkinter import filedialog
from matplotlib import animation
from PIL import ImageTk, Image
import tkinter.ttk as ttk
from tktooltip import ToolTip
from matplotlib.patches import Ellipse
import numpy as np
from matplotlib.widgets import RectangleSelector
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
from skimage import io
from skimage.filters import gaussian
from skimage import feature
from skimage.segmentation import active_contour
customtkinter.set_appearance_mode("light")  # Modes: system (default), light, dark
customtkinter.set_default_color_theme("green")  # Themes: blue (default), dark-blue, green
class Application(tk.Frame):
    def __init__(self, master=None):
        super().__init__(master)
        self.master = master
        self.master.title("Segmentacja Obrazów")
        self.master.geometry("800x1000")
        self.master.maxsize(800,1000)
        self.pack(fill='both', expand=True)
        self.create_widgets()
       
    def create_widgets(self):
        # Create label for canvas
        self.canvas_label =customtkinter.CTkLabel(self, text="Segmentacja Obrazów Metodą Aktywnego Konturu",font=customtkinter.CTkFont(size=20, weight="bold"))
        self.canvas_label.pack(pady=10)
        # Create canvas to display image
        self.figure = Figure(figsize=(5, 5), dpi=100, facecolor='#11ba71')
        self.ax = self.figure.add_subplot(111)
        self.ax.set_xticks([]), self.ax.set_yticks([])
        self.canvas = FigureCanvasTkAgg(self.figure, master=self)
        self.canvas.get_tk_widget().pack(padx=10, pady=10)
        self.canvas.mpl_connect('motion_notify_event', self.update_xy_label)

        # Create label to display x and y parameters
        self.xy_label = customtkinter.CTkLabel(self, text="")
        self.xy_label.pack()


        # Create open button to select image
        self.open_button =  customtkinter.CTkButton(self, text="Wybierz Obraz", command=self.load_image)
        self.open_button.pack(pady=10)
        self.open_button.place(x=175, y=570)
        ToolTip(self.open_button, msg="Wybierz obraz do segmentacji", follow=True)
        # Create run button to segment image
        self.run_button = customtkinter.CTkButton(self, text="Segmentuj Obraz", command=self.segment_image)
        self.run_button.pack(pady=10)
        self.run_button.place(x=485, y=570)
        ToolTip(self.run_button, msg="Naciśnij do przeprowadzenia segmentacji", follow=True)
        
        self.run_button = customtkinter.CTkButton(self, text="Model 3D", command=self.make_3image)
        self.run_button.pack(pady=10)
        self.run_button.place(x=485, y=600)
        ToolTip(self.run_button, msg="Naciśnij do wyświetlenia modelu 3D", follow=True)
        # Create label and entry for alpha parameter
        self.alpha_label = customtkinter.CTkLabel(self, text="Parametr Alpha:")
        self.alpha_label.pack()
        self.alpha_entry = customtkinter.CTkEntry(self)
        self.alpha_entry.insert(0, "0.015")
        self.alpha_entry.pack()
        ToolTip(self.alpha_entry, msg="Parametr Alpha", follow=True)
        # Create label and entry for beta parameter
        self.beta_label = customtkinter.CTkLabel(self, text="Parametr Beta:")
        self.beta_label.pack()
        self.beta_entry = customtkinter.CTkEntry(self)
        self.beta_entry.insert(0, "10")
        self.beta_entry.pack()
        ToolTip(self.beta_entry, msg="Parametr Beta", follow=True)
        # Create label and entry for gamma parameter
        self.gamma_label = customtkinter.CTkLabel(self, text="Parametr Gamma:")
        self.gamma_label.pack()
        self.gamma_entry =  customtkinter.CTkEntry(self)
        self.gamma_entry.insert(0, "0.001")
        self.gamma_entry.pack()
        ToolTip(self.gamma_entry, msg="Parametr Gamma", follow=True)
        # Create label and entry for x coordinate of initial contour
        self.init_x_label = customtkinter.CTkLabel(self, text="Pozycja Y początkowego konturu:")
        self.init_x_label.pack()
        self.init_x_entry =  customtkinter.CTkEntry(self)
        self.init_x_entry.insert(0, "250")
        self.init_x_entry.pack()
        ToolTip(self.init_x_entry, msg="Punkt x początkowy naszego konturu(canvas ma wymiary 500x500)", follow=True)
        # Create label and entry for y coordinate of initial contour
        self.init_y_label = customtkinter.CTkLabel(self, text="Pozycja X początkowego konturu:")
        self.init_y_label.pack()
        self.init_y_entry =  customtkinter.CTkEntry(self)
        self.init_y_entry.insert(0, "250")
        self.init_y_entry.pack()
        ToolTip(self.init_y_entry, msg="Punkt y początkowy naszego konturu(canvas ma wymiary 500x500)", follow=True)
        # Create label and entry for radius
        self.init_rX_label = customtkinter.CTkLabel(self, text="Wartość promienia X:")
        self.init_rX_label.pack()
        self.init_rX_entry =  customtkinter.CTkEntry(self)
        self.init_rX_entry.insert(0, "100")
        self.init_rX_entry.pack()
        ToolTip(self.init_rX_entry, msg="Wartośc promienia-obszar jaki zaznaczamy do konturu(canvas ma wymiary 500x500)", follow=True)
        # Create label and entry for radius
        self.init_rY_label = customtkinter.CTkLabel(self, text="Wartość promienia Y:")
        self.init_rY_label.pack()
        self.init_rY_entry =  customtkinter.CTkEntry(self)
        self.init_rY_entry.insert(0, "100")
        self.init_rY_entry.pack()
        ToolTip(self.init_rY_entry, msg="Wartośc promienia-obszar jaki zaznaczamy do konturu(canvas ma wymiary 500x500)", follow=True)



    

    def make_3image(self):
        settings = pymeshlab.MeshSetScript()

        # load images from specified directory
        settings.load_meshes_from_directory("C:/Users/Lenovo/OneDrive/Pulpit/aom/aa")

        # specify size of 3D image element
        settings.set_attribute("voxelSize", [1, 1, 5])

        # create voxel object from the series of images
        volume = pymeshlab.MeshSet()
        volume.load_from_script(settings)

        # define ISO value to build surface
        iso = 127.0

        # convert voxel object to mesh
        mesh = volume.surface_voxelization(iso)

        # save mesh to .stl file
        stl_path = "C:/Users/Lenovo/OneDrive/Pulpit/aom/mesh.stl"
        mesh.save_mesh(stl_path)

        # load the mesh
        trimesh_mesh = trimesh.load_mesh(stl_path)

        # display the mesh
        trimesh_mesh.show()
    def update_xy_label(self, event):
        x, y = event.x, event.y
        self.xy_label.configure(text=f"x={x}, y={y}")

    def load_image(self):
        # Open file dialog to select image
        file_path = filedialog.askopenfilename()

        # Read DICOM file using pydicom
        if file_path.endswith('.dcm'):
            dicom_file = pydicom.dcmread(file_path)

            # Convert DICOM pixel array to numpy array and normalize to 0-255 range
            image_array = dicom_file.pixel_array
            image_array = (image_array - np.min(image_array)) / (np.max(image_array) - np.min(image_array)) * 255
            image_array = image_array.astype(np.uint8)

            # Apply Canny filter to image
            image_array = feature.canny(gaussian(image_array, sigma=2), sigma=3)


        # Convert numpy array to PIL Image and resize to fit canvas
        self.image = Image.fromarray(image_array)
        self.image = self.image.resize((500, 500))

        # Display image on canvas
        self.ax.clear()
        self.ax.set_xticks([]), self.ax.set_yticks([])
        self.ax.imshow(self.image, cmap=plt.cm.gray)
        self.canvas.draw()

    def segment_image(self):
        # Check if an image has been loaded
        if not hasattr(self, 'image'):
            print("Please load an image first.")
            return

        # Convert image to numpy array
        image_array = np.array(self.image)      

        # Preprocess the image by applying a Gaussian filter:
        image_gaus = gaussian(image_array, sigma=3)
        image_can = feature.canny(image_gaus, sigma=3)
        # Define the initial contour:
        init_rX = int(self.init_rX_entry.get())
        init_rY = int(self.init_rY_entry.get())
        init_y = int(self.init_y_entry.get())
        init_x = int(self.init_x_entry.get())
        s = np.linspace(0, 2*np.pi, 400)
        x = init_x+ init_rX*np.cos(s)
        y = init_y +  init_rY*np.sin(s)
        init = np.array([x, y]).T
        # Set up active contour parameters
        alpha = float(self.alpha_entry.get())
        beta = float(self.beta_entry.get())
        gamma = float(self.gamma_entry.get())


        # Run active contour algorithm
        snake = active_contour(gaussian(image_can,4, preserve_range=False),init, alpha=alpha, beta=beta, gamma=gamma, max_num_iter=500)


        # Display segmented image on canvas
        self.ax.clear()
        self.ax.set_xticks([]), self.ax.set_yticks([])
        self.ax.imshow(image_array, cmap=plt.cm.gray)
        self.ax.plot(init[:, 1], init[:, 0], '--r', lw=3)
        self.ax.plot(snake[:, 1], snake[:, 0], '-b', lw=3)
        self.ax.add_artist(Ellipse((init_x, init_y), 5, 5, edgecolor='b', facecolor='r'))
        self.canvas.draw()


root = tk.Tk()
app = Application(master=root)
app.mainloop()

KeyboardInterrupt: 

: 